<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-准备数据" data-toc-modified-id="1-准备数据-1">1 准备数据</a></span></li><li><span><a href="#2-pulp解决整数线性规划最大化问题" data-toc-modified-id="2-pulp解决整数线性规划最大化问题-2">2 pulp解决整数线性规划最大化问题</a></span><ul class="toc-item"><li><span><a href="#2.1-构建整数ILP实例" data-toc-modified-id="2.1-构建整数ILP实例-2.1">2.1 构建整数ILP实例</a></span></li><li><span><a href="#2.2-定义变量" data-toc-modified-id="2.2-定义变量-2.2">2.2 定义变量</a></span></li><li><span><a href="#2.3-定义目标函数" data-toc-modified-id="2.3-定义目标函数-2.3">2.3 定义目标函数</a></span></li><li><span><a href="#2.4-设定约束条件" data-toc-modified-id="2.4-设定约束条件-2.4">2.4 设定约束条件</a></span></li><li><span><a href="#2.5-求解" data-toc-modified-id="2.5-求解-2.5">2.5 求解</a></span></li><li><span><a href="#2.6-查看最优解和最优值" data-toc-modified-id="2.6-查看最优解和最优值-2.6">2.6 查看最优解和最优值</a></span><ul class="toc-item"><li><span><a href="#2.6.1-打印最优解和最优值" data-toc-modified-id="2.6.1-打印最优解和最优值-2.6.1">2.6.1 打印最优解和最优值</a></span></li><li><span><a href="#2.6.2--将最优解转为DataFrame" data-toc-modified-id="2.6.2--将最优解转为DataFrame-2.6.2">2.6.2  将最优解转为DataFrame</a></span></li></ul></li></ul></li><li><span><a href="#3-总结：" data-toc-modified-id="3-总结：-3">3 总结：</a></span></li></ul></div>

一个农民承包了6块耕地共300亩，准备播种小麦，玉米，水果和蔬菜四种农产品，已知各种农产品的计划播种面积、每块土地种植不同农产品的单产收益    
如何进行合理安排，使得总收益最大    
可以使用pulp/ortools工具    

In [52]:
import pulp
import ortools
import numpy as np
import pandas as pd

# 1 准备数据

In [2]:
# 每种作物的单位收益 (元/亩)和计划播种面积
df_incomes = pd.read_csv("farming_unit_gains.csv")

# 每个地块的面积
df_area = pd.read_csv("farming_area.csv")

In [3]:
df_incomes

,产物,地块1,地块2,地块3,地块4,地块5,地块6,计划播种面积（亩）
0,小麦,500,550,630,1000,800,700,76
1,玉米,800,700,600,950,900,930,88
2,水果,1000,960,840,650,600,700,96
3,蔬菜,1200,1040,980,860,880,780,40


In [4]:
df_area

,地块,地块面积（亩）
0,地块1,42
1,地块2,56
2,地块3,44
3,地块4,39
4,地块5,60
5,地块6,59


In [5]:
df_incomes['计划播种面积（亩）'].sum(), df_area['地块面积（亩）'].sum()

(300, 300)

# 2 pulp解决整数线性规划最大化问题
## 2.1 构建整数ILP实例

In [6]:
prob = pulp.LpProblem(name="MaxIncomeOfFarmer", sense=pulp.LpMaximize)

## 2.2 定义变量

In [7]:
# 产物种类数，土地种类数
n_crops = len(df_incomes)
n_lands = len(df_area)

variables = []

# x产物地块
for i in range(n_crops):
    var = []
    for j in range(n_lands):
        # 整数变量类型
        var.append(pulp.LpVariable(name=f'x{i}{j}', lowBound=0, cat='Integer'))
    variables.append(var)
        
print(variables)

[[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]


In [8]:
# 产物4种， 地块6种
len(variables), len(variables[0])

(4, 6)

In [9]:
# 将定义好的变量转成数组
variables = np.array(variables)
variables

array([[x00, x01, x02, x03, x04, x05],
       [x10, x11, x12, x13, x14, x15],
       [x20, x21, x22, x23, x24, x25],
       [x30, x31, x32, x33, x34, x35]], dtype=object)

In [24]:
type(variables[0, 0])

pulp.pulp.LpVariable

## 2.3 定义目标函数

In [10]:
df_incomes.loc[:, '地块1':'地块6']

,地块1,地块2,地块3,地块4,地块5,地块6
0,500,550,630,1000,800,700
1,800,700,600,950,900,930
2,1000,960,840,650,600,700
3,1200,1040,980,860,880,780


In [11]:
# 构造目标函数
prob += pulp.lpDot(df_incomes.loc[:, '地块1':'地块6'].values, variables)

In [12]:
prob.objective # 查看目标函数

500*x00 + 550*x01 + 630*x02 + 1000*x03 + 800*x04 + 700*x05 + 800*x10 + 700*x11 + 600*x12 + 950*x13 + 900*x14 + 930*x15 + 1000*x20 + 960*x21 + 840*x22 + 650*x23 + 600*x24 + 700*x25 + 1200*x30 + 1040*x31 + 980*x32 + 860*x33 + 880*x34 + 780*x35 + 0

## 2.4 设定约束条件

In [13]:
n_crops = len(df_incomes)
n_lands = len(df_area)

for i in range(n_crops):
    # 产物i 计划种植的面积
    plan_area = df_incomes.iloc[i, -1]
    prob += pulp.lpSum(variables[i]) == plan_area
    
for j in range(n_lands):
    # 地块j 可种植的总面积
    land_area = df_area.iloc[j, -1]
    prob += pulp.lpSum(variables[:, j]) == land_area

In [14]:
prob.constraints # 查看约束条件

OrderedDict([('_C1', 1*x00 + 1*x01 + 1*x02 + 1*x03 + 1*x04 + 1*x05 + -76 = 0),
             ('_C2', 1*x10 + 1*x11 + 1*x12 + 1*x13 + 1*x14 + 1*x15 + -88 = 0),
             ('_C3', 1*x20 + 1*x21 + 1*x22 + 1*x23 + 1*x24 + 1*x25 + -96 = 0),
             ('_C4', 1*x30 + 1*x31 + 1*x32 + 1*x33 + 1*x34 + 1*x35 + -40 = 0),
             ('_C5', 1*x00 + 1*x10 + 1*x20 + 1*x30 + -42 = 0),
             ('_C6', 1*x01 + 1*x11 + 1*x21 + 1*x31 + -56 = 0),
             ('_C7', 1*x02 + 1*x12 + 1*x22 + 1*x32 + -44 = 0),
             ('_C8', 1*x03 + 1*x13 + 1*x23 + 1*x33 + -39 = 0),
             ('_C9', 1*x04 + 1*x14 + 1*x24 + 1*x34 + -60 = 0),
             ('_C10', 1*x05 + 1*x15 + 1*x25 + 1*x35 + -59 = 0)])

## 2.5 求解

In [18]:
prob.solve()

1

## 2.6 查看最优解和最优值

In [20]:
# 查看解的状态
pulp.LpStatus[prob.status]

'Optimal'

### 2.6.1 打印最优解和最优值

In [32]:
# 最优值
max_income = pulp.value(prob.objective)
max_income

284230.0

In [33]:
print(f"获得最大收益{max_income}时，4种产物在各地块的种植面积如下：")
for i in range(n_crops):
    for j in range(n_lands):
        crop = df_incomes.loc[i, '产物']
        land = df_area.loc[j, '地块']
        val = variables[i, j].value()
        print("产物:{} 在土地:{} 中种植面积为:{}".format(crop, land, val))

获得最大收益284230.0时，4种产物在各地块的种植面积如下：
产物:小麦 在土地:地块1 中种植面积为:0.0
产物:小麦 在土地:地块2 中种植面积为:0.0
产物:小麦 在土地:地块3 中种植面积为:6.0
产物:小麦 在土地:地块4 中种植面积为:39.0
产物:小麦 在土地:地块5 中种植面积为:31.0
产物:小麦 在土地:地块6 中种植面积为:0.0
产物:玉米 在土地:地块1 中种植面积为:0.0
产物:玉米 在土地:地块2 中种植面积为:0.0
产物:玉米 在土地:地块3 中种植面积为:0.0
产物:玉米 在土地:地块4 中种植面积为:0.0
产物:玉米 在土地:地块5 中种植面积为:29.0
产物:玉米 在土地:地块6 中种植面积为:59.0
产物:水果 在土地:地块1 中种植面积为:2.0
产物:水果 在土地:地块2 中种植面积为:56.0
产物:水果 在土地:地块3 中种植面积为:38.0
产物:水果 在土地:地块4 中种植面积为:0.0
产物:水果 在土地:地块5 中种植面积为:0.0
产物:水果 在土地:地块6 中种植面积为:0.0
产物:蔬菜 在土地:地块1 中种植面积为:40.0
产物:蔬菜 在土地:地块2 中种植面积为:0.0
产物:蔬菜 在土地:地块3 中种植面积为:0.0
产物:蔬菜 在土地:地块4 中种植面积为:0.0
产物:蔬菜 在土地:地块5 中种植面积为:0.0
产物:蔬菜 在土地:地块6 中种植面积为:0.0


### 2.6.2  将最优解转为DataFrame

In [35]:
df_solution = pd.DataFrame(variables, index=df_incomes['产物'], columns=df_area['地块'])
df_solution

地块,地块1,地块2,地块3,地块4,地块5,地块6
产物,,,,,,
小麦,x00,x01,x02,x03,x04,x05
玉米,x10,x11,x12,x13,x14,x15
水果,x20,x21,x22,x23,x24,x25
蔬菜,x30,x31,x32,x33,x34,x35


In [39]:
df_solution = df_solution.apply(lambda x: x.apply(lambda y: y.value()))

In [40]:
df_solution

地块,地块1,地块2,地块3,地块4,地块5,地块6
产物,,,,,,
小麦,0.0,0.0,6.0,39.0,31.0,0.0
玉米,0.0,0.0,0.0,0.0,29.0,59.0
水果,2.0,56.0,38.0,0.0,0.0,0.0
蔬菜,40.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df_solution.sum(1), df_solution.sum()

(产物
 小麦    76.0
 玉米    88.0
 水果    96.0
 蔬菜    40.0
 dtype: float64,
 地块
 地块1    42.0
 地块2    56.0
 地块3    44.0
 地块4    39.0
 地块5    60.0
 地块6    59.0
 dtype: float64)

In [51]:
# 将给定的Lp问题写入.lp文件
prob.writeLP('最大收益LP.lp')

[x00,
 x01,
 x02,
 x03,
 x04,
 x05,
 x10,
 x11,
 x12,
 x13,
 x14,
 x15,
 x20,
 x21,
 x22,
 x23,
 x24,
 x25,
 x30,
 x31,
 x32,
 x33,
 x34,
 x35]

# 3 总结：

+ 变量可以转为数组，方便给变量设定约束条件
+ pulp.lpDot()的v1可以是数组类型，但不支持DataFrame类型；
+ 设定约束条件时，最后需要使用的是逻辑运算符，比如：相等需要用 == 表示，一个=会报错；
+ LpProblem实例属性支持查看设定好的目标函数、约束条件、求解状态；
+ for循环打印最优解的方法，觉得看着不够直观，所以尝试了将变量的数组转成DataFrame，并apply成变量的最优解，看起来更清晰。通过这次尝试，证明DataFrame是个包容性很强的容器，可以存各种类型的元素。